In [1]:
import sys

sys.path.append("../")

In [2]:
import requests
import time
import pandas as pd
import numpy as np
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from stats_arb.tests import adf_test, kpss_test, cal_half_life, pp_test
from datetime import datetime, timedelta
from ta.volatility import BollingerBands

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6) # (w, h)
plt.ioff()

In [3]:
API_BASE = 'https://fapi.binance.com/fapi/v1/'
# DATA_PATH = '/mnt/c/Users/vmodg/source/repos/BeanRepos/crypto-pair-trading/data/'
DATA_PATH = '/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/crypto/'
TIMEFRAME = '1d'
lookback = 365

TIMEFRAME = '1h'
lookback = 30*4

# TIMEFRAME = '5m'
# lookback = 20


In [4]:


LABELS = [
    'open_time',
    'open',
    'high',
    'low',
    'close',
    'volume',
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]

DROP_COLUMNS=[
    'close_time',
    'quote_asset_volume',
    'number_of_trades',
    'taker_buy_base_asset_volume',
    'taker_buy_quote_asset_volume',
    'ignore'
]


def get_batch(symbol, interval='1m', start_time=0, limit=400):
    """Use a GET request to retrieve a batch of candlesticks. Process the JSON into a pandas
    dataframe and return it. If not successful, return an empty dataframe.
    """

    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': limit
    }
    try:
        # timeout should also be given as a parameter to the function
        response = requests.get(f'{API_BASE}klines', params, timeout=30)
    except requests.exceptions.ConnectionError:
        print('Connection error, Cooling down for 5 mins...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    except requests.exceptions.Timeout:
        print('Timeout, Cooling down for 5 min...')
        time.sleep(15)
        return get_batch(symbol, interval, start_time, limit)

    if response.status_code == 200:
        return pd.DataFrame(response.json(), columns=LABELS)
    
    print(f'Got erroneous response back {symbol}: {response}. {response.text}')
    return pd.DataFrame([])


def get_candles(base, quote, start_date: datetime, end_date=None, interval='1m'):
    batches = []

    if end_date is None:
        end_date = datetime.utcnow()

    last_timestamp = int(start_date.timestamp()) * 1000
    # gather all candlesticks available, starting from the last timestamp loaded from disk or 0
    # stop if the timestamp that comes back from the api is the same as the last one
    previous_timestamp = None

    while previous_timestamp != last_timestamp:
        # stop if we reached data from today
        if datetime.fromtimestamp(last_timestamp / 1000) >= end_date:
            break

        previous_timestamp = last_timestamp

        new_batch = get_batch(
            symbol=base + quote,
            interval=interval,
            start_time=last_timestamp
        )

        # requesting candles from the future returns empty
        # also stop in case response code was not 200
        if new_batch.empty:
            break

        last_timestamp = new_batch['open_time'].max()

        # sometimes no new trades took place yet on date.today();
        # in this case the batch is nothing new
        if previous_timestamp == last_timestamp:
            break

        batches.append(new_batch)
        last_datetime = datetime.fromtimestamp(last_timestamp / 1000)

        covering_spaces = 20 * ' '
        print(datetime.now(), base, quote, interval, str(last_datetime) + covering_spaces, end='\r', flush=True)

    if len(batches) > 0:
        # write clean version of csv to parquet
        df = pd.concat(batches, ignore_index=True)
        df.drop(columns=DROP_COLUMNS, inplace=True)
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.set_index('open_time', inplace=True)
        df = df[~df.index.duplicated(keep='first')]
        return df



In [5]:
# import requests

# url = "https://fapi.binance.com/fapi/v1/exchangeInfo"

# payload={}
# headers = {}

# response = requests.request("GET", url, headers=headers, data=payload).json()

# symbols = [s['symbol'] for s in response['symbols'] if s['contractType'] == 'PERPETUAL' and s['quoteAsset'] == 'USDT']
# # symbols[100:]
# symbols = symbols[:50]
# len(symbols)


In [6]:
# symbols = pd.read_csv(f'/mnt/d/Working/PersonalProjects/Trading/trading-agent/crypto-pair-trading/data/symbols.csv')['symbol'].values.tolist()
symbols = ['BTCUSDT',
'ETHUSDT',
'BCHUSDT',
'XRPUSDT',
'LTCUSDT',
'ETCUSDT',
'LINKUSDT',
'XLMUSDT',
'ADAUSDT',
'BNBUSDT',
'ATOMUSDT',
'ALGOUSDT',
'DOTUSDT',
'SOLUSDT',
'AVAXUSDT',
'MATICUSDT',
'XMRUSDT',
'NEARUSDT',
'AVAXUSDT',
'ATOMUSDT',
'MANAUSDT',
'UNIUSDT',
'BCHUSDT']
symbols = [s.replace('USDT', '') for s in symbols]

data = []
start_time = datetime.utcnow() - timedelta(days=lookback)

for symbol in symbols:
    df = pd.read_csv(f'{DATA_PATH}/{TIMEFRAME}/{symbol}-USDT.csv', parse_dates=['open_time'], index_col=['open_time'])
    df = df[df.index >= start_time].copy()
    # print('start time', df.index[-1].to_pydatetime())

    df1 = get_candles(
        base=symbol, 
        quote='USDT', 
        start_date=df.index[-1].to_pydatetime(),
        # end_date=datetime.now() - timedelta(days=15),
        interval=TIMEFRAME
    )
    if df1 is not None:
        df = pd.concat([df, df1])
        df = df[~df.index.duplicated(keep='first')]

    df.rename(columns={'close': symbol}, inplace=True)
    # the data is too long, just limit to recent period
    log = np.log(df[symbol].astype(np.float32))
    data.append(log)

df = pd.concat(data, axis=1)
df = df.dropna(axis=1, how='all')
# df.dropna(inplace=True, how='any')

# our of sample
# df = df[df.index <= datetime.now() - timedelta(days=15)].copy()

df.tail()

,BTC,ETH,BCH,XRP,LTC,ETC,LINK,XLM,ADA,BNB,...,SOL,AVAX,MATIC,XMR,NEAR,AVAX,ATOM,MANA,UNI,BCH
open_time,,,,,,,,,,,,,,,,,,,,,
2023-04-28 21:00:00,10.285687,7.546119,4.768903,-0.737935,4.494015,2.979806,1.951750,-2.357992,-0.904856,5.780157,...,3.145445,2.861057,-0.003005,5.038640,0.647103,2.861057,2.457450,-0.599111,1.706202,4.768903
2023-04-28 22:00:00,10.286274,7.545929,4.769158,-0.740448,4.495467,2.982394,1.951466,-2.358309,-0.905351,5.779694,...,3.145961,2.863514,-0.001902,5.035003,0.648673,2.863514,2.462150,-0.598929,1.707290,4.769158
2023-04-28 23:00:00,10.285343,7.544401,4.769582,-0.735220,4.495132,2.983103,1.951040,-2.357464,-0.905846,5.780157,...,3.149783,2.864484,-0.002102,5.031353,0.649718,2.864484,2.460443,-0.597655,1.708559,4.769582
2023-04-29 00:00:00,10.283703,7.542977,4.769158,-0.735637,4.492449,2.981684,1.944910,-2.360744,-0.907083,5.777590,...,3.147294,2.860199,-0.006823,5.028999,0.651804,2.860199,2.458135,-0.600750,1.706020,4.769158
2023-04-29 01:00:00,10.283836,7.543903,4.769158,-0.737935,4.494797,2.983255,1.947623,-2.360956,-0.908571,5.774241,...,3.148196,2.860657,-0.005314,5.029326,0.650761,2.860600,2.459418,-0.598929,1.705657,4.769243


In [7]:
p = 1
COINTEGRATION_CONFIDENCE_LEVEL = 90

# the 90%, 95%, and 99% confidence levels for the trace statistic and maximum 
# eigenvalue statistic are stored in the first, second, and third column of 
# cvt and cvm, respectively
confidence_level_cols = {
    90: 0,
    95: 1,
    99: 2
}
confidence_level_col = confidence_level_cols[COINTEGRATION_CONFIDENCE_LEVEL]


def test_johansen(symbol_pairs):
    df_t = df[symbol_pairs].copy()
    df_t.dropna(inplace=True)

    # The second and third parameters indicate constant term, with a lag of 1. 
    result = coint_johansen(df_t, 0, p)

    trace_crit_value = result.cvt[:, confidence_level_col]
    eigen_crit_value = result.cvm[:, confidence_level_col]
#     print("trace_crit_value",trace_crit_value)
#     print("eigen_crit_value",eigen_crit_value)
#     print("lr1",result.lr1)
#     print("lr2",result.lr2)

    # The trace statistic and maximum eigenvalue statistic are stored in lr1 and lr2;
    # see if they exceeded the confidence threshold
    if np.all(result.lr1 >= trace_crit_value) and np.all(result.lr2 >= eigen_crit_value):
        # print(f"{symbol_pairs} are cointegrated")
        # The first i.e. leftmost column of eigenvectors matrix, result.evec, contains the best weights.
        v1= result.evec[:,0:1]
        hr=v1/-v1[1] #to get the hedge ratio divide the best_eigenvector by the negative of the second component of best_eigenvector
        #the regression will be: close of symbList[1] = hr[0]*close of symbList[0] + error
        #where the beta of the regression is hr[0], also known as the hedge ratio, and
        #the error of the regression is the mean reverting residual signal that you need to predict, it is also known as the "spread"
        #the spread = close of symbList[1] - hr[0]*close of symbList[0] or alternatively (the same thing):
        #do a regression with close of symbList[0] as x and lose of symbList[1] as y, and take the residuals of the regression to be the spread.
        coint_pair = dict(hedge_ratio=v1[:, 0])
        for i, s in enumerate(symbol_pairs):
            coint_pair[f'sid_{i+1}'] = s

        cointegrating_pairs.append(coint_pair)



In [8]:
import traceback


def calculate_spread(df, coint_df, selected_row, hedge_ratio, nb_symbols=2):
    spread = None
    for i in range(nb_symbols):
        if spread is None:
            spread = df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
        else:
            spread += df[coint_df[f'sid_{i + 1}'].iloc[selected_row]] * hedge_ratio[i]
    
    spread.dropna(inplace=True)
    return spread


critical_val = 0.05

def find_stationary_portfolio(coint_df):
    data = []
    for i, _ in coint_df.iterrows():
        try:
            hedge_ratio = coint_df.iloc[i]['hedge_ratio']
            _df = df.copy()
            spread = calculate_spread(_df, coint_df, i, hedge_ratio)
            p_val = adf_test(spread, verbose=False)
            if p_val < critical_val:
                half_life = cal_half_life(spread)
                pairs_name = coint_df[[col for col in coint_df.columns if 'sid' in col]].iloc[i].values
                # print(i, pairs_name, 'is stationary with half life', half_life)
                # print(' ')
                data.append({
                    'i': i,
                    'pairs': pairs_name,
                    'half_life': half_life
                })
        except:
            print(coint_df.iloc[i])
            # traceback.print_exc()
    
    return pd.DataFrame(data)


# find_stationary_portfolio(coint_df)

In [9]:

import itertools as it

nb_symbols = 2
cointegrating_pairs = []

#get symbol pairs
pairs = list(it.combinations(symbols, nb_symbols))

for pair in pairs:
    try:
        test_johansen(list(pair))
    except Exception:
        pass

coint_df = pd.DataFrame(cointegrating_pairs)
coint_df['i'] = coint_df.index
coint_df.head()

,hedge_ratio,sid_1,sid_2,i
0,"[0.6073921513562598, 11.0235067892563]",LTC,ETC,0
1,"[17.88059467237473, -6.757412282702286]",LTC,ADA,1
2,"[1.9233849279037079, 4.25040287055366]",LTC,SOL,2
3,"[12.0756095922089, -4.493418376948825]",LTC,XMR,3
4,"[8.234574486699676, 4.08769089116854]",ETC,LINK,4


In [10]:
coint_df.to_csv(f'coint_df_{TIMEFRAME}.csv')


# Result

In [11]:
import warnings

warnings.filterwarnings("ignore")

stationary_df = find_stationary_portfolio(coint_df)
stationary_df = pd.merge(stationary_df, coint_df, on='i')
stationary_df.dropna(inplace=True)
stationary_df['i'] = stationary_df.i.astype('int64')
stationary_df['half_life'] = stationary_df.half_life.astype('int64')
if len(stationary_df) > 0:
    stationary_df.sort_values(by=['half_life'], inplace=True)
# stationary_df


In [12]:
stationary_df.to_csv(f'stationary_df_{TIMEFRAME}.csv', index=False)
stationary_df

,i,pairs,half_life,hedge_ratio,sid_1,sid_2
25,26,"[ALGO, UNI]",43,"[21.541007587564557, -35.868435530687634]",ALGO,UNI
12,12,"[ETC, NEAR]",62,"[23.249510953797298, -9.370381319700083]",ETC,NEAR
16,16,"[LINK, ADA]",72,"[24.884300381377834, -12.526588998211242]",LINK,ADA
32,36,"[NEAR, MANA]",77,"[18.1315474482863, -13.22532530004431]",NEAR,MANA
18,18,"[LINK, DOT]",81,"[25.063345302290223, -13.429322446692677]",LINK,DOT
13,13,"[ETC, MANA]",81,"[19.159631626995708, -5.7811720489466385]",ETC,MANA
7,7,"[ETC, ALGO]",83,"[16.7041924535499, -4.9566700824834635]",ETC,ALGO
11,11,"[ETC, XMR]",85,"[16.362725025293784, -15.12756943737181]",ETC,XMR
1,1,"[LTC, ADA]",97,"[17.88059467237473, -6.757412282702286]",LTC,ADA
8,8,"[ETC, DOT]",97,"[14.658778944992985, -3.1690609511129324]",ETC,DOT
